<h2> MINeD Hackathon 2024 </h2>

<h4> <b> Track 2 Revelio Labs </b> <br>
<b> Track Name: </b> PeopleMetrics: Shaping the Next-Gen Workforce </h4>

<b> Team Name: </b> Omega <br>
<b> Team Members:</b> <br>
&nbsp; Parv Thacker (Leader) <br>
&nbsp; Richa Yadav <br>
&nbsp; Khushi Patel <br>
&nbsp; Niti Patel <br>
&nbsp; Jaival Chhag <br> <br>

In [1]:
import docx2txt
from PyPDF2 import PdfReader
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import re
import os
import csv
import warnings
warnings.filterwarnings("ignore")       ## To Remove warnings for rare symbols that are missing from certain fonts (for saving)

In [2]:
def extract_from_docx(file_path):
    # Extract text from DOCX file
    text = docx2txt.process(file_path)
    return text

def extract_from_pdf(file_path):
    # Extract text from PDF file
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        for pageNo in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[pageNo]
            text += page.extract_text()
    return text

def create_resume_dict(text):
    # Create a dictionary of resume sections based on headings
    headings = ['Project Experience','Skills Summary', 'Key Projects', 'Awards', 'Introduction', 'Technical skills', 'Experience', 'Education', 'Leadership and Achievements', 'Career Experience', 'Education & Certifications', 'License', 'Additional Skills' ]  # Add more headings as needed
    resume_dict = {}
    for heading in headings:
        if heading.lower() in text.lower():
            
            start_index = text.lower().index(heading.lower())
            if headings.index(heading) == len(headings) - 1:
                end_index = len(text)
            else:
                next_heading_index = text.lower().find(headings[headings.index(heading) + 1].lower(), start_index)
                end_index = next_heading_index if next_heading_index != -1 else len(text)
                
            section_text = text[start_index:end_index]
            resume_dict[heading] = section_text
    return resume_dict

def extract_contact_info(text):
    # Extract name, email, phone number, and LinkedIn URL using regular expressions
    
    name_match = re.search(r'[A-Za-z]+ [A-Za-z]+', text)
    email_match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text)
    phone_number_match = re.search(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})', text)
    linkedin_match = re.search(r'https?://www\.linkedin\.com/[\w\d-]+', text)
    
    name = name_match.group(0) if name_match else "Not mentioned"
    email = email_match.group(0) if email_match else "Not mentioned"
    phone_number = phone_number_match.group(0) if phone_number_match else "Not mentioned"
    linkedin_url = linkedin_match.group(0) if linkedin_match else "Not mentioned"
    
    return name, email, phone_number, linkedin_url

def shortlist_resumes(resumes, shortlisting_words):
    # Match terms in resume to identify ones that match the requirements (Match any one)
    shortlisted_resumes = []
    for i, resume in enumerate(resumes):
        resume_shortlisted = False
        for section_heading, section_text in resume.items():
            if isinstance(section_text, list):
                continue
            for word in shortlisting_words:
                if word.lower() in section_text.lower():
                    resume_shortlisted = True
                    break
            if resume_shortlisted:
                break
        if resume_shortlisted:
            shortlisted_resumes.append(resume)
            print("Shortlisted:", i)

    return shortlisted_resumes

In [3]:
def match_words(main_string, word_list):
    main_string = main_string.lower()                   # for case-insensitive matching
    word_list = [word.lower() for word in word_list]    # convert word list to lowercase

    # Split the main string into words
    main_words = main_string.split()

    # Empty list to store matching words
    matching_words = []

    # Exact matches with words from the list
    for word in main_words:
        if word in word_list:
            matching_words.append(word)

    # Print the matching words
    matches = []
    for match in matching_words:
        matches += [match]
        
    return list(set(matches))

# List of words to match
word_list = [
    "Programming", "Algorithms", "Data Structures", "Databases", "web development",
    "Networking", "Cybersecurity", "Cloud Computing", "Machine Learning", "Operating Systems",
    "Problem Solving", "Communication", "Collaboration", "Adaptability", "Creativity",
    "Critical Thinking", "Debugging", "Automation", "Version Control", "Analysis",
    "Automation", "DevOps", "Scripting", "Design", "Optimization", "Trouble shooting",
    "Scalability", "Performance", "Documentation", "Integration", "Testing", "Security",
    "Agility", "Persistence", "Leadership", "Innovation", "Research", "Modularity",
    "Efficiency", "Resilience", "Visualization", "Abstraction", "Parallelism",
    "Multithreading", "Parallelism", "Time Management", "Interdisciplinary", "Analytical",
    "UX/UI", "Versioning", "Debugging", "Flexibility", "Creativity", "Adaptability",
    "Precision", "Initiative", "Resourcefulness", "Empathy", "Collaboration", "Determination",
    "Communication", "Organization", "Leadership", "Attention to detail", "Problem solving",
    "Time Management", "Critical Thinking", "Adaptability", "Resilience", "Decision Making"
]

In [4]:
def create_title_mapping(csv_file):
    title_mapping = {}
    with open(csv_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            reported_title = row['Reported Job Title']
            standardized_title = row['Title']
            onet_soc_code = row['O*NET-SOC Code']
            title_mapping[reported_title] = {'title': standardized_title, 'code': onet_soc_code}
    return title_mapping

# Default location of the csv file
csv_file = 'onet.csv'               # Update the locations of ONET here
title_mapping = create_title_mapping(csv_file)

# Function to standardize job titles using the mapping
def standardize_job_titles(text, title_mapping):
    standardized_titles = []
    # Sort reported job titles by length in descending order
    sorted_titles = sorted(title_mapping.keys(), key=len, reverse=True)
    for reported_title in sorted_titles:
        if reported_title.lower() in text.lower():
            # If the reported title is not part of a longer reported title already included in standardized_titles
            if not any(reported_title.lower() in title[0].lower() for title in standardized_titles):
                standardized_title = title_mapping[reported_title]['title']
                onet_soc_code = title_mapping[reported_title]['code']
                standardized_titles.append([reported_title, standardized_title, onet_soc_code])
                # Remove shorter variations of the matched reported job title from the text
                text = text.replace(reported_title.lower(), '', 1)
    return standardized_titles

In [5]:
# Convert lists to string to store it directly in unstructured/semistructured formats for easy viewing
def convert_to_string(lst):
    if isinstance(lst, list):
        if all(isinstance(item, list) for item in lst):
            return '\n'.join(', '.join(sub) for sub in lst)
        else:
            return ', '.join(lst)
    else:
        return lst

In [6]:
# Sample shortlisting words
shortlisting_words = ['Data Science', 'Data Scientist']         # Set a selection criteria, if any here

# Sample resumes (file paths)
resume_files = []
resume_files = os.listdir("./resumes/")
for i, file in enumerate(resume_files):
    resume_files[i] = "./resumes/"+file

In [7]:
# Crop for explaination
resume_files = resume_files

In [11]:
if __name__ == "__main__":
    # Extract text from resumes
    resumes = []
    for file in resume_files:
        if file.endswith('.docx'):
            text = extract_from_docx(file)
        elif file.endswith('.pdf'):
            text = extract_from_pdf(file)
            
        # Call the match_words function with the main string and word list
        skillsex = match_words(text, word_list)
        # print(skillsex)
            
        resume_dict = create_resume_dict(text)
        name, email, phone_number, linkedin_url = extract_contact_info(text)
        
        resume_dict["FileNm"] = file[10:-4]
        resume_dict["Name"] = name
        resume_dict["Email"] = email
        resume_dict["Phone"] = phone_number
        resume_dict["LinkedIn"] = linkedin_url
        resume_dict["Skills_Extracted"] = skillsex
        resume_dict["standardized_titles"] = standardize_job_titles(text, title_mapping)
        resumes.append(resume_dict)


    # Shortlist resumes based on shortlisting words
    shortlisted_resumes = shortlist_resumes(resumes, shortlisting_words)
    # shortlisted_resumes = resumes

    # Save shortlisted resumes as images and PDFs
    for idx, resume in enumerate(shortlisted_resumes, 1):
        # Create a new PDF for each shortlisted resume
        pdf_file_path = f"shortlisted_resume_{resume['FileNm']}.pdf"
        c = canvas.Canvas(pdf_file_path, pagesize=letter)
        
        # Write contact information at the top
        c.drawString(100, 770, f"Name: {resume['Name']}")
        c.drawString(100, 750, f"Email: {resume['Email']}")
        c.drawString(100, 730, f"Phone: {resume['Phone']}")
        if resume['LinkedIn']:
            c.linkURL(resume['LinkedIn'], f"LinkedIn: {resume['LinkedIn']}")
        
        # Write resume content to the PDF
        y_coordinate = 710
        for heading, section_text in resume.items():
            section_text = convert_to_string(section_text)
            if heading not in ["Name", "Email", "Phone", "LinkedIn"]:
                c.drawString(100, y_coordinate, heading)
                c.drawString(100, y_coordinate - 20, section_text)
                y_coordinate -= 40
        
        c.save()
            
        # Save resume as image
        image_file_path = f"shortlisted_resume_{resume['FileNm']}.png"
        plt.figure(figsize=(8, 6))
        plt.text(0.5, 0.5, "\n".join([f"{heading}:\n{convert_to_string(section_text)}\n" for heading, section_text in resume.items()]), ha='center', va='center', wrap=True)
        plt.axis('off')
        plt.savefig(image_file_path, bbox_inches='tight')
        plt.close()
        
        # Save resume as text file
        txt_file_path = f"shortlisted_resume_{resume['FileNm']}.txt"
        with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write("\n".join([f"{heading}:\n{convert_to_string(section_text)}\n" for heading, section_text in resume.items()]))
        
        print(f"Shortlisted Resume {idx} saved as {pdf_file_path}, {image_file_path}, and {txt_file_path}")

Shortlisted: 1
Shortlisted: 3
Shortlisted: 4
Shortlisted: 6
Shortlisted: 8
Shortlisted: 9
Shortlisted: 10
Shortlisted: 11
Shortlisted: 12
Shortlisted: 13
Shortlisted: 14
Shortlisted: 15
Shortlisted: 16
Shortlisted: 17
Shortlisted: 18
Shortlisted: 19
Shortlisted: 20
Shortlisted: 21
Shortlisted: 22
Shortlisted: 23
Shortlisted: 24
Shortlisted: 26
Shortlisted: 27
Shortlisted: 28
Shortlisted: 29
Shortlisted: 30
Shortlisted: 31
Shortlisted: 32
Shortlisted: 33
Shortlisted: 34
Shortlisted: 35
Shortlisted: 36
Shortlisted: 37
Shortlisted: 38
Shortlisted: 39
Shortlisted: 40
Shortlisted: 41
Shortlisted: 42
Shortlisted: 43
Shortlisted: 44
Shortlisted: 45
Shortlisted: 48
Shortlisted: 49
Shortlisted: 50
Shortlisted: 51
Shortlisted: 52
Shortlisted: 53
Shortlisted: 54
Shortlisted: 56
Shortlisted: 57
Shortlisted: 58
Shortlisted: 65
Shortlisted: 70
Shortlisted Resume 1 saved as shortlisted_resume_documents20220826-1-v01cla.pdf, shortlisted_resume_documents20220826-1-v01cla.png, and shortlisted_resume_doc

Sortlisted resumes are saved in the files in various formats. The data extracted is shown. ONET standardization and skills are extracted.

In [9]:
shortlisted_resumes[0]

{'Technical skills': 'Technical Skills\nDatabases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB\nData Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI\nCloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure\n(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive\n',
 'Experience': 'Experience\nClark Associates Inc Pennsylvania, USA\nFinancial Systems and Data Intern May 2022 - Aug 2022\n•Extracted data from varied data sources to automate Summary Income financial reports(monthly, quarterly and\nyearly) for the various subsidiaries of Clark Associates Inc. that reduced manual entry time by 70%.\n•Collaborated with cross-functional teams to enhance and optimize existing Power BI reports that helped stakeholders\nanalyze the financial performance of their business.\nDacapo Brokerage India Private Limited Mumbai,India\nSenior Analyst Aug 2020 - Jul 2021\n•Created visualiza

In [10]:
import pandas as pd
df = pd.DataFrame(shortlisted_resumes)

combines = [
    ['Project Experience', 'Key Projects',],
    ['Technical skills','Skills Summary',],
    ['Experience', 'Career Experience', 'Education & Certifications',],
]

for group in combines:
    representative = group[0]
    present_columns = [col for col in group if col in df.columns]
    if present_columns:
        df[representative] = df[present_columns].apply(lambda x: ' & '.join(x.dropna().astype(str)), axis=1)
        df.drop(present_columns[1:], axis=1, inplace=True)
    
    
orderColumns = ['FileNm', 'Name', 'Email', 'Phone', 'LinkedIn', 'Skills_Extracted', 'standardized_titles'] + [col for col in df.columns if col not in ['FileNm', 'Name', 'Email', 'Phone', 'LinkedIn', 'Skills_Extracted', 'standardized_titles']]
df = df[orderColumns]

df.to_csv('extracted.csv', index=False)
df

,FileNm,Name,Email,Phone,LinkedIn,Skills_Extracted,standardized_titles,Technical skills,Experience,Education,Leadership and Achievements,Project Experience,Introduction,Awards
0,documents20220826-1-v01cla,Ashwini Kalidas,ashwini.vedula@nyu.edu,616-1347,Not mentioned,"[leadership, databases, analysis, performance,...","[[Software Engineer, Computer Numerically Cont...","Technical Skills\nDatabases & Tools : MySQL, S...","Experience\nClark Associates Inc Pennsylvania,...","Education\nNew York University, New York, NY S...",Leadership and Achievements\n•Conferred Quarte...,,NaN,NaN
1,documents20220830-1-eq72bl,ANAM IQBA,anami@andrew.cmu.edu,(646) 207-4431,Not mentioned,"[automation, efficiency, analysis, testing, al...","[[Research Associate, Remote Sensing Technicia...",,EXPERIENCE ...,EDUCATION \nCARNE GIE MELLON UNIVERSITY (C...,NaN,PROJECT EXPERIENCE \n Disaster Tweet Predi...,"Introduction to Deep Learning, Big Data & Larg...",NaN
2,DS Updated 2,ASachin Asokan,asokansachin73@gmail.com,7654071650,Not mentioned,"[analysis, algorithms, precision, performance,...","[[Industrial Engineer, Industrial Engineers, 1...",,EXPERIENCE\nProject Lead\n• Spearheaded a team...,EDUCATION\nIndustrial Engineering\nPurdue Univ...,NaN,,NaN,NaN
3,Maaz khan _ d17,Maaz Muhammad,maazkhan1211@gmail.com,Not mentioned,Not mentioned,"[leadership, analysis, testing, algorithms, co...","[[Business Intelligence Analyst, Business Inte...",,Experience \nBusiness Intelligence Analyst \D...,Education \nM.S. Computer Science (GPA 3. 77)...,NaN,,NaN,NaN
4,NYU CDS-Revelio Labs Resume Book_Fall 2022 Car...,Baipeng Gong,Not mentioned,(646)877-8364,Not mentioned,"[analysis, research, performance, optimization...","[[President, Chief Executives, 11-1011.00], [A...",,"EXPERIENCE \nMorgan Stanley New York, NY \n...",EDUCATION \nNew York University September 2...,NaN,,"Introduction to Data Science (A/B Testing),...",NaN
5,NYU CDS-Revelio Labs Resume Book_Fall 2022 Car...,Qi Dong,qd2046@nyu.edu,6468661808,Not mentioned,"[research, analysis, testing, precision, perfo...","[[Industrial Engineer, Industrial Engineers, 1...",,EXPERIENCE \nLoad Forecasting & Analytics Inte...,EDUCATION \nNew York University May 2023 \nMa...,NaN,,"Introduction to Data Science, Programming for ...",NaN
6,NYU CDS-Revelio Labs Resume Book_Fall 2022 Car...,New York,soomin.kim@nyu.edu,(929) 837-0434,Not mentioned,"[leadership, databases, research, analysis, al...","[[Data Analyst, Statistical Assistants, 43-911...","TECHNICAL SKILLS\nLanguages:\nPython (pandas, ...","EXPERIENCE\nAttentive\n, New York, NY\nSep\n–\...",EDUCATION\nNew York University\nMay 2023\nM.S....,NaN,,NaN,NaN
7,NYU CDS-Revelio Labs Resume Book_Fall 2022 Car...,WORK EXPERIENCE,tl3493@nyu.edu,Not mentioned,Not mentioned,"[efficiency, leadership, databases, research, ...","[[Research Assistant, Bioinformatics Technicia...",,EXPERIENCE \nData Analytics Intern \nSpectru...,EDUCATION \nM.S. \nData Science \nNew York ...,NaN,,NaN,NaN
8,NYU CDS-Revelio Labs Resume Book_Fall 2022 Car...,New York,vanessaziweixu@gmail.com,929-988-3639,https://www.linkedin.com/in,"[efficiency, analysis, performance, design, pr...","[[Graduate Assistant, Graduate Teaching Assist...","Technical Skills: Shell. Git, Python (PyTorch,...",EXPERIENCE ...,EDUCATION ...,NaN,,NaN,NaN
9,NYU CDS-Revelio Labs Resume Book_Fall 2022 Car...,Xinge Hu,xh1051@nyu.edu,(617) 331-5722,https://www.linkedin.com/in,"[research, analysis, performance, visualizatio...","[[Research Analyst, Statistical Assistants, 43...",,EXPERIENCE\nIntellipro Group ...,EDUCATION\nNEW YORK UNIVERSITY ...,NaN,,NaN,NaN
